# DataFrame Import

This file extracts an existing the f1db from the Postgres dump provided by the F1DB project. This file then saves a handful of pickled DataFrames to be used in other notebooks

In [181]:
import json
import sqlalchemy
import pandas as pd

Give access to notebook for Postgres Database actions

In [182]:
# Replace these with your actual credentials
db_username = "postgres"
db_password = input("Database Password: ")
db_host = "localhost"
db_port = "5432"
db_name = "f1db"


In [183]:
# Connect to postgres database
engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

## Import Info

### Table Look Up Functionality
Create a function that can be used to look up specific tables, or list all table

In [184]:
schema_info_path = "resources/f1db/"
schema_file = "f1db.schema.json"

with open(schema_info_path + schema_file, 'r') as f:
    info_json = json.load(f)

schema_names = list(dict(info_json["properties"].items()).keys())
schema_descriptions = [x["description"] for x in list(dict(info_json["properties"].items()).values())]

schema_info = dict(zip(schema_names, schema_descriptions))

def get_schema_info(table : str = ""):
    if table in schema_info.keys():
        print(table)
        print(schema_info[table])
        print()
    else:
        for table in schema_info.keys():
            get_schema_info(table)

# ex:
# get_schema_info()

### Column Look Up Functionality
Create an object that holds information of all schema descriptions and data types (accessed using dot notation)


In [185]:
# Placeholder class that is used to attached attribute names of varying aliases
class Null:
    def __str__(self):
        return "\n".join(f"{k} = {v}" for k, v in vars(self).items()) + "\n"


In [189]:
column_info = info_json['definitions']

with engine.connect() as c:
    query = c.execute(sqlalchemy.text(""
                                       "SELECT table_name "
                                       "FROM information_schema.tables "
                                       "WHERE table_schema = 'public' "
                                       "    AND table_type = 'BASE TABLE';"))
    table_names = query.all()

table_names = [list(i)[0] for i in table_names]

table_var_names = list(map(lambda x: x.replace('_',' ').title().replace(' ',''), table_names))

schema = Null()

for table in table_var_names:
        try:
            attr_names = list(column_info[table]['properties'].keys())
            exec(f"schema.{table} = Null()")
            for attr_name in attr_names:
                exec(f"schema.{table}.{attr_name} = Null()")

                try:
                    attr_type = column_info[table]['properties'][attr_name]["type"]
                    exec(f"schema.{table}.{attr_name}.type = attr_type")
                except KeyError:
                    pass

                try:
                    attr_desc = column_info[table]['properties'][attr_name]["description"]
                    exec(f"schema.{table}.{attr_name}.description = attr_desc")
                except KeyError:
                    pass
        except KeyError:
            pass

# ex:
# print(schema.Continent.id)

## Pickle Tables

In [191]:
for table in table_names:
    with engine.connect() as conn:
        query = f"SELECT * FROM {table}"

        dataFrame = pd.read_sql_query(query, conn)
        dataFrame.to_pickle(f'pickled/{table}.pkl')

In [179]:
get_schema_info()

drivers
The list of drivers, each representing the drivers who participated in races, including biographical details and statistics.

constructors
The list of constructors, each representing the teams or manufacturers responsible for designing and producing race cars, including statistics.

chassis
The list of chassis, each representing a specific race car chassis design.

engineManufacturers
The list of engine manufacturers, each representing the entities producing engines, including statistics.

engines
The list of engines, each representing the specific engines used in race cars, including configuration, performance data, and manufacturers.

tyreManufacturers
The list of tyre manufacturers, each representing the companies that supply tyres, including statistics.

entrants
The list of entrants, each representing the teams and individuals who have participated in races.

circuits
The list of circuits, each representing a specific racing track, including geographical location and race 